<a href="https://colab.research.google.com/github/BCI-and-Neuroergonomics-Lab/3D-MB-CNN/blob/dev/KT_HPTuning.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Mount Drive


In [ ]:
# Loading your drive so CoLab has access to it
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


# Imports

In [ ]:
import tensorflow as tf
from psutil import virtual_memory

from tensorflow.keras.models import Model
from tensorflow.keras.layers import Dense, Activation, Dropout, Softmax
from tensorflow.keras.layers import Conv3D, SpatialDropout3D, AveragePooling3D
from tensorflow.keras.layers import Conv2D, SpatialDropout2D, AveragePooling2D
from tensorflow.keras.layers import DepthwiseConv2D, SeparableConv2D
from tensorflow.keras.layers import BatchNormalization
from tensorflow.keras.layers import Input, Flatten, Add
from tensorflow.keras.constraints import max_norm

import numpy as np
from sklearn.model_selection import KFold, train_test_split
import matplotlib.pyplot as plt
from sklearn.metrics import confusion_matrix, ConfusionMatrixDisplay

from IPython import display

from uuid import uuid4

try:
  from pandas import DataFrame
except ModuleNotFoundError:
  !pip install pandas
  from pandas import DataFrame

try:
  import kerastuner as kt
except ModuleNotFoundError:
  !pip install keras-tuner --upgrade
  import kerastuner as kt


     |████████████████████████████████| 133 kB 5.1 MB/s 


/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:32: DeprecationWarning: `import kerastuner` is deprecated, please use `import keras_tuner`.


# Check For GPU Presence

In [ ]:
%tensorflow_version 2.x

USE_GPU = True

if (USE_GPU):
  device_name = tf.test.gpu_device_name()
  if device_name != '/device:GPU:0':
    raise SystemError('GPU device not found')
  print('Found GPU at: {}'.format(device_name))

  gpu_info = !nvidia-smi
  gpu_info = '\n'.join(gpu_info)
  if gpu_info.find('failed') >= 0:
    print('Not connected to a GPU')
  else:
    print(gpu_info)
    
  ram_gb = virtual_memory().total / 1e9
  print('Your runtime has {:.1f} gigabytes of available RAM\n'.format(ram_gb))

Found GPU at: /device:GPU:0
Thu Apr 28 11:55:55 2022       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 460.32.03    Driver Version: 460.32.03    CUDA Version: 11.2     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla P100-PCIE...  Off  | 00000000:00:04.0 Off |                    0 |
| N/A   41C    P0    34W / 250W |    375MiB / 16280MiB |      1%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                            

# Constants


In [ ]:
#@markdown #Debugging Information

VERBOSE = 1 #@param [0, 1] {type:"raw"}

#@markdown #High-Level Model Configuration
MODE = "Motor" #@param ["Emotion", "Motor"]
DIMENSION = '3D' #@param ["2D", "3D"]
BRANCHED = 'MB' #@param ["MB", "SRF", "MRF", "LRF"]

#@markdown #Low-Level Model Configuration
#@markdown ## Model Training Configuration
# Options are 'sparse_categorical_crossentropy', 'mean_absolute_error', 'mean_squared_error', or 'categorical_crossentropy'
# Can be anything Keras accepts, above are just examples
#@markdown Loss function used for model training
LOSS_FUNCTION = 'categorical_crossentropy' #@param ['categorical_crossentropy', 'mean_absolute_error', 'sparse_categorical_crossentropy', 'mean_squared_error']

# Options are 'Adam' or 'SGD'
#@markdown Optimizer used for model training
OPTIMIZER_OPTION = "Adam" #@param ["Adam", "SGD"]

EPOCHS = 500 #@param {type:"integer"}

#@markdown ## Model Type Independent Hyperparameters
#@markdown In this section, please enter type None for any variables that should not be declared, any values entered will be fixed during tuning

BATCH_SIZE = None #@param ["None"] {type:"raw", allow-input: true}
INIT_LR = None #@param ["None"] {type:"raw", allow-input: true}

#@markdown ## 2D Model Hyperparameter Confiuration
#@markdown In this section, please enter type None for any variables that should not be declared, any values entered will be fixed during tuning

# Parameters for EEGNet2D
D_2D =  None#@param ["None"] {type:"raw", allow-input: true}
SRF_2D =  None#@param ["None"] {type:"raw", allow-input: true}
MRF_2D =  None#@param ["None"] {type:"raw", allow-input: true}
LRF_2D =  None#@param ["None"] {type:"raw", allow-input: true}
F1_2D = None #@param ["None"] {type:"raw", allow-input: true}
F2_2D =  None#@param ["None"] {type:"raw", allow-input: true}
CONV_SIZE_2D =  None#@param ["None"] {type:"raw", allow-input: true}
DROPOUT_RATE_2D =  0.5#@param ["None"] {type:"raw", allow-input: true}
DROPOUT_TYPE_2D =  "Dropout"#@param ["None", "Dropout", "SpatialDropout2D"] {type:"string", allow-input: true}
if (DROPOUT_TYPE_2D == "None"):
  DROPOUT_TYPE_2D = None
NORM_RATE_2D =  0.25#@param ["None"] {type:"raw", allow-input: true}
POOLING_1_2D =  None#@param ["None"] {type:"raw", allow-input: true}
POOLING_2_2D =  None#@param ["None"] {type:"raw", allow-input: true}
#@markdown ## 3D Model Hyperparameter Confiuration
#@markdown In this section, please enter type None for any variables that should not be declared, any values entered will be fixed during tuning

# Parameters for EEGNet3D
D_3D = 4#@param ["None"] {type:"raw", allow-input: true}
SRF_3D = None #@param ["None"] {type:"raw", allow-input: true}
MRF_3D = None #@param ["None"] {type:"raw", allow-input: true}
LRF_3D = None #@param ["None"] {type:"raw", allow-input: true}
F1_3D = 16#@param ["None"] {type:"raw", allow-input: true}
F2_3D = 4#@param ["None"] {type:"raw", allow-input: true}
CONV_SIZE_3D = None#@param ["None"] {type:"raw", allow-input: true}
DROPOUT_RATE_3D = 0.5#@param ["None"] {type:"raw", allow-input: true}
DROPOUT_TYPE_3D = 'Dropout'#@param ["None"] {type:"string", allow-input: true}
if (DROPOUT_TYPE_3D == "None"):
  DROPOUT_TYPE_3D = None
NORM_RATE_3D = 0.25#@param ["None"] {type:"raw", allow-input: true}
POOLING_1_3D = None#@param ["None"] {type:"raw", allow-input: true}
POOLING_2_3D = None#@param ["None"] {type:"raw", allow-input: true}
SPATIAL_SIZE_3D = 4#@param ["None"] {type:"raw", allow-input: true}


#@markdown #Data Settings
#@markdown Type of data used for each split between test, train, and validation
TRAIN_PROCESSING_STAGE = "SHUFFLED" #@param ["SHUFFLED", "AVERAGED", "CROPPED", "PROCESSED"]
TEST_PROCESSING_STAGE = "SHUFFLED" #@param ["SHUFFLED", "AVERAGED", "CROPPED", "PROCESSED"]
VAL_PROCESSING_STAGE = "SHUFFLED" #@param ["SHUFFLED", "AVERAGED", "CROPPED", "PROCESSED"]

#@markdown Number of classes
NUM_CLASSES = 4 #@param [2, 3, 4] {type:"raw"}
#@markdown Number of samples in each datapoint
DEFAULT_SAMPLES = 313 #@param {type:"integer"}
#@markdown Number of subjects in the Emotion dataset (SEED IV)
EMOTION_NUM_SUBJECTS = 15#@param {type:"integer"}
#@markdown Number of subjects in the Motor dataset (BCICIV_2a)
MOTOR_NUM_SUBJECTS = 9#@param {type:"integer"}

#@markdown Subject to use for emotion analysis
EMOTION_SUBJECT_NUM = 1#@param {type:"integer"}
#@markdown Subject to use for motor analysis
MOTOR_SUBJECT_NUM = 7#@param {type:"integer"}
if (MODE == "Motor"):
  SUBJECT_NUM = MOTOR_SUBJECT_NUM
elif (MODE == "Emotion"):
  SUBJECT_NUM = EMOTION_SUBJECT_NUM

# Results in a shuffle following data creation and when the Kfolds are generated
#@markdown Whether or not to shuffle data <br>
#@markdown Does not shuffle between splits
SHUFFLE = True #@param {type:"boolean"}


if (BRANCHED == 'MB'):
  BRANCHES = ("SRF", "MRF", "LRF")
elif (BRANCHED == 'SRF'):
  BRANCHES = ("SRF", )
elif (BRANCHED == 'MRF'):
  BRANCHES = ("MRF", )
elif (BRANCHED == 'LRF'):
  BRANCHES = ("LRF", )

# ---- Bayesian Variables ------------------------------------------------------
#@markdown #Bayesian Tuning Parameters
NUM_ITERATIONS = 256 #@param {type:"integer"}
NUM_CV = 1 #@param ["None", 1, 2, 3, 4, 5] {type:"raw"}
NUM_JOBS = 1 #@param {type:"integer"}
NUM_POINTS = 1 #@param {type:"integer"}
OVERWRITE = False #@param {type:"boolean"}
SEARCH = True #@param {type:"boolean"}
PROJECT_TAG = "BranchPooling" #@param {type:"string"}
if (PROJECT_TAG != ""):
  PROJECT_TAG += "-"

# ---- Directory Stuff ---------------------------------------------------------
#@markdown #Data Directories
TUNING_DIRECTORY = "/content/drive/MyDrive/BCI-Lab-Drive/Tuning" #@param {type:"string"}

EMOTION_PROCESSED_DIR = "/content/drive/MyDrive/BCI-Lab-Drive/Datasets/SEED/SEED_IV_processed" #@param {type:"string"}
EMOTION_CROPPED_DIR = "/content/drive/MyDrive/BCI-Lab-Drive/Datasets/SEED/SEED_IV_cropped" #@param {type:"string"}
EMOTION_AVERAGED_DIR = "/content/drive/MyDrive/BCI-Lab-Drive/Datasets/SEED/SEED_IV_averaged" #@param {type:"string"}
EMOTION_SHUFFLED_DIR = "/content/drive/MyDrive/BCI-Lab-Drive/Datasets/SEED/SEED_IV_shuffled" #@param {type:"string"}

EMOTION_DIR_DICT = {"PROCESSED": EMOTION_PROCESSED_DIR, "CROPPED": EMOTION_CROPPED_DIR, "AVERAGED": EMOTION_AVERAGED_DIR, "SHUFFLED": EMOTION_SHUFFLED_DIR}

MOTOR_PROCESSED_DIR = "/content/drive/MyDrive/BCI-Lab-Drive/Datasets/BCICIV_2a/BCICIV_2a_processed" #@param {type:"string"}
MOTOR_CROPPED_DIR = "/content/drive/MyDrive/BCI-Lab-Drive/Datasets/BCICIV_2a/BCICIV_2a_cropped" #@param {type:"string"}
MOTOR_AVERAGED_DIR = "/content/drive/MyDrive/BCI-Lab-Drive/Datasets/BCICIV_2a/BCICIV_2a_averaged" #@param {type:"string"}
MOTOR_SHUFFLED_DIR = "/content/drive/MyDrive/BCI-Lab-Drive/Datasets/BCICIV_2a/BCICIV_2a_shuffled" #@param {type:"string"}

MOTOR_DIR_DICT = {"PROCESSED": MOTOR_PROCESSED_DIR, "CROPPED": MOTOR_CROPPED_DIR, "AVERAGED": MOTOR_AVERAGED_DIR, "SHUFFLED": MOTOR_SHUFFLED_DIR}

PROCESSED_SUFFIX = "_processed.npy" #@param {type:"string"}
CROPPED_SUFFIX = "_cropped.npy" #@param {type:"string"}
AVERAGED_SUFFIX = "_averaged.npy" #@param {type:"string"}
SHUFFLED_SUFFIX = "_shuffled.npy" #@param {type:"string"}

SUFFIX_DICT = {"PROCESSED": PROCESSED_SUFFIX, "CROPPED": CROPPED_SUFFIX, "AVERAGED": AVERAGED_SUFFIX, "SHUFFLED": SHUFFLED_SUFFIX}

TRAIN_DIR_APPEND = "train/" #@param {type:"string"}
TEST_DIR_APPEND = "test/" #@param {type:"string"}
VAL_DIR_APPEND = "val/" #@param {type:"string"}

if (MODE == 'Emotion'):
  try:
    TRAIN_DIR = EMOTION_DIR_DICT[TRAIN_PROCESSING_STAGE]
    TRAIN_SUFFIX = SUFFIX_DICT[TRAIN_PROCESSING_STAGE]

    TEST_DIR = EMOTION_DIR_DICT[TEST_PROCESSING_STAGE]
    TEST_SUFFIX = SUFFIX_DICT[TEST_PROCESSING_STAGE]

    VAL_DIR = EMOTION_DIR_DICT[VAL_PROCESSING_STAGE]
    VAL_SUFFIX = SUFFIX_DICT[VAL_PROCESSING_STAGE]

  except KeyError:
    raise Exception("Invalid Data Processing Stage, please specify 'PROCESSED', 'CROPPED', 'AVERAGED', or 'SHUFFLED'.")

  DISPLAY_LABELS = ['Neutral', 'Sad', 'Fear', 'Happy']

elif (MODE == 'Motor'):
  try:
    TRAIN_DIR = MOTOR_DIR_DICT[TRAIN_PROCESSING_STAGE]
    TRAIN_SUFFIX = SUFFIX_DICT[TRAIN_PROCESSING_STAGE]

    TEST_DIR = MOTOR_DIR_DICT[TEST_PROCESSING_STAGE]
    TEST_SUFFIX = SUFFIX_DICT[TEST_PROCESSING_STAGE]

    VAL_DIR = MOTOR_DIR_DICT[VAL_PROCESSING_STAGE]
    VAL_SUFFIX = SUFFIX_DICT[VAL_PROCESSING_STAGE]
    
  except KeyError:
    raise Exception("Invalid Data Processing Stage, please specify 'PROCESSED', 'CROPPED', 'AVERAGED', or 'SHUFFLED'.")

  DISPLAY_LABELS = ['Left', 'Right', 'Foot', 'Tongue']

else:
  raise Exception("Invalid Data Mode, please specify 'Emotion' or 'Motor'.")

if (OPTIMIZER_OPTION == 'Adam'):
  OPTIMIZER = tf.keras.optimizers.Adam
elif (OPTIMIZER_OPTION == 'SGD'):
  OPTIMIZER = tf.keras.optimizers.SGD
else:
  raise Exception("Invalid Optimizer Option, please specify 'Adam' or 'SGD'.")
  
DIM_3D_SUFFIX = "_3d/"
DIM_2D_SUFFIX = "_2d/"

if (DIMENSION == '3D'):
  TRAIN_DIR = TRAIN_DIR + DIM_3D_SUFFIX + TRAIN_DIR_APPEND
  TEST_DIR = TEST_DIR + DIM_3D_SUFFIX + TEST_DIR_APPEND
  VAL_DIR = VAL_DIR + DIM_3D_SUFFIX + VAL_DIR_APPEND
elif (DIMENSION == '2D'):
  TRAIN_DIR = TRAIN_DIR + DIM_2D_SUFFIX + TRAIN_DIR_APPEND
  TEST_DIR = TEST_DIR + DIM_2D_SUFFIX + TEST_DIR_APPEND
  VAL_DIR = VAL_DIR + DIM_2D_SUFFIX + VAL_DIR_APPEND
else:
  raise Exception("Invalid Dimension, please specify '3D' or '2D'.")

# ------------------------------------------------------------------------------

# Metrics

#@markdown #Metrics

ACCURACY_METRIC = True #@param {type:"boolean"}
PRECISION_METRIC = True #@param {type:"boolean"}
RECALL_METRIC = True #@param {type:"boolean"}

#@markdown #Callbacks

# Epoch End Callback
#@markdown Custom epoch end message
EPOCH_END = False #@param {type:"boolean"}

# Profiling Callback
#@markdown TensorBoard Profiling
PROFILING = False #@param {type:"boolean"}
PROFILING_OUT_DIR = "/content/drive/MyDrive/BCI-Lab-Drive/Output/TensorBoard/"#@param {type:"string"}

# Early Stopping Callback
#@markdown Early Stopping
EARLY_STOPPING = True #@param {type:"boolean"}
EARLY_STOPPING_PATIENCE = 50#@param {type:"integer"}

# Model Checkpointing Callback
#@markdown Model Checkpointing
MODEL_CHECKPOINTING = False #@param {type:"boolean"}
CHECKPOINT_OUT_DIR = "/content/drive/MyDrive/BCI-Lab-Drive/Output/Checkpoints/"#@param {type:"string"}
CHECKPOINT_OUT_DIR += str(uuid4())
print("Saving Checkpointed Models to:\n" + CHECKPOINT_OUT_DIR)

# Live Plot
#@markdown Live Training Plot
# Supported values include 'Loss', 'Accuracy', and 'F1-Score'
# These values will override the above metrics, as they must be included for plot
LIVE_PLOTTING = False #@param {type:"boolean"}
PLOT_METRIC = 'Accuracy' #@param ["Accuracy", "Loss", "F1-Score"] {type:"string"}
BAD_PLOT_METRIC = "Unimplemented Plot Metric, please check the value of constant 'PLOT_METRIC'."




Saving Checkpointed Models to:
/content/drive/MyDrive/BCI-Lab-Drive/Output/Checkpoints/801a0d19-eaa8-4524-ad33-6d941c74779b


# Callbacks & Metrics

## Metrics

In [ ]:
METRICS = []
METRIC_NAMES = []

if (ACCURACY_METRIC or PLOT_METRIC == 'Accuracy'):
  METRICS.append(tf.keras.metrics.CategoricalAccuracy(name='Accuracy'))
  METRIC_NAMES.append('Accuracy')

if (PRECISION_METRIC or PLOT_METRIC == 'F1-Score'):
  METRICS.append(tf.keras.metrics.Precision(name='Precision'))
  METRIC_NAMES.append('Precision')

if (RECALL_METRIC or PLOT_METRIC == 'F1-Score'):
  METRICS.append(tf.keras.metrics.Recall(name='Recall'))
  METRIC_NAMES.append('Recall')

## Callbacks 

### Keras Callbacks

In [ ]:
class ModifiedEpochEnd(tf.keras.callbacks.Callback):
  def __init__(self, metrics=('loss', 'acc'), metric_names=('Loss', 'Accuracy')):
    self.metrics = metrics
    self.metric_names = metric_names

  def on_epoch_end(self, epoch, logs=None):
    print("Finished epoch {}:".format(epoch+1))

    print("{:<12} {:<10} {:<10}".format("Metric", "Training", "Validation"))
    for metric_name in self.metric_names:
      print("{:<12} {:<10.3f} {:<10.3f}".format(metric_name, logs[metric_name], logs['val_'+metric_name]))
    print("") # Forcing newline
  
  def on_train_end(self, logs=None):
    print("\nTraining Finished")

class LessInfoEpochEnd(tf.keras.callbacks.Callback):
  def __init__(self):
    self.best_epoch = None
    self.best_loss = None
    self.best_val_loss = None

  def on_epoch_end(self, epoch, logs=None):
    val_loss = logs['val_loss']
    if (self.best_val_loss is None or val_loss < self.best_val_loss):
      self.best_epoch = epoch+1
      self.best_loss = logs['loss']
      self.best_val_loss = val_loss
    output = "\rFinished Epoch {}: Loss/Val - {:.3}/{:.3}\tBest Epoch {}: Loss/Val - {:.3}/{:.3}".format(epoch+1, logs['loss'], logs['val_loss'], self.best_epoch, self.best_loss, self.best_val_loss)
    print(output, end='')

class PlotLosses(tf.keras.callbacks.Callback):
    def on_train_begin(self, logs={}):
        self.epoch_num = 1
        self.x = []
        self.plot_metric = []
        self.plot_val_metric = []

        self.best_epoch = None
        self.best_val_metric = None
        self.best_metric = None

        self.hdisplay = display.display("", display_id=True)
        tmp = plt.ion()

        self.fig = plt.figure()
        self.ax = self.fig.add_subplot(111)
        self.ax.set_xlabel('Epoch')
        self.ax.set_ylabel(PLOT_METRIC)
        self.ax.set_title("Training Performance")
        self.caption = plt.figtext(0.5, -0.1, '', wrap=True, horizontalalignment='center', fontsize=12)
        plt.ylim((0, 1))
        
        self.logs = []

    def on_epoch_end(self, epoch, logs={}):
        self.logs.append(logs)
        self.x.append(self.epoch_num)
        if (PLOT_METRIC == 'Accuracy'):
          metric = logs.get('Accuracy')
          val_metric = logs.get('val_Accuracy')
          if (self.best_val_metric is None or val_metric > self.best_val_metric):
            self.best_epoch = self.epoch_num
            self.best_metric = metric
            self.best_val_metric = val_metric
          self.plot_metric.append(metric)
          self.plot_val_metric.append(val_metric)
        elif (PLOT_METRIC == 'Loss'):
          metric = logs.get('loss')
          val_metric = logs.get('val_loss')
          if (self.best_val_metric is None or val_metric < self.best_val_metric):
            self.best_epoch = self.epoch_num
            self.best_metric = metric
            self.best_val_metric = val_metric
          self.plot_metric.append(metric)
          self.plot_val_metric.append(val_metric)
        elif (PLOT_METRIC == 'F1-Score'):
          metric = self.compute_f1(logs.get('Precision'), logs.get('Recall'))
          val_metric = self.compute_f1(logs.get('val_Precision'), logs.get('val_Recall'))
          if (self.best_val_metric is None or val_metric > self.best_val_metric):
            self.best_epoch = self.epoch_num
            self.best_metric = metric
            self.best_val_metric = val_metric
          self.plot_metric.append(metric)
          self.plot_val_metric.append(val_metric)
        else:
          raise Exception(BAD_PLOT_METRIC)
        self.epoch_num += 1

        caption_txt = "Best Epoch\nEpoch {} - {}: {:.3} - Val {}: {:.3}".format(self.best_epoch, PLOT_METRIC, self.best_metric, PLOT_METRIC, self.best_val_metric)
        self.caption.set(text=caption_txt)
        self.ax.plot(self.x, self.plot_metric, 'r', label=PLOT_METRIC if epoch == 0 else "")
        self.ax.plot(self.x, self.plot_val_metric, 'b', label="Val " + PLOT_METRIC if epoch == 0 else "")
        self.fig.legend()

        self.hdisplay.update(self.fig)
      
    def on_train_end(self, logs):
        plt.close()

    def compute_f1(self, precision, recall):
      if (precision == 0 and recall == 0):
        return 0
      return (2*precision*recall)/(precision+recall)

EpochEndCallback = LessInfoEpochEnd()
TensorBoardCallback = tf.keras.callbacks.TensorBoard(log_dir=PROFILING_OUT_DIR)
EarlyStoppingCallback = tf.keras.callbacks.EarlyStopping(patience=EARLY_STOPPING_PATIENCE, verbose=VERBOSE)
ModelCheckpointCallback = tf.keras.callbacks.ModelCheckpoint(CHECKPOINT_OUT_DIR, verbose=VERBOSE, save_best_only=True, save_weights_only=True)

CALLBACKS = []
if (EPOCH_END):
    CALLBACKS.append(LessInfoEpochEnd)
if (PROFILING):
    CALLBACKS.append(TensorBoardCallback)
if (EARLY_STOPPING):
    CALLBACKS.append(EarlyStoppingCallback)
if (MODEL_CHECKPOINTING):
    CALLBACKS.append(ModelCheckpointCallback)
if (LIVE_PLOTTING):
    CALLBACKS.append(PlotLosses())

### Skopt Callbacks


In [ ]:
# callback handler
def bayes_callback(optim_result):
    score = -optim_result['fun']
    print("\nBest Score: %s\n" % score)
    if score >= 0.90:
        print('Model Achieved Accuracy of 90%, stopping')
        return True

# Load Data

In [ ]:
def load_data(data_dir, data_suff, subject):
  x = np.load(data_dir + "A0" + str(subject) + "D" + data_suff, encoding='latin1', allow_pickle=True)
  y = np.load(data_dir + "A0" + str(subject) + "K" + data_suff, encoding='latin1', allow_pickle=True)
  return x, y

def select_classes(X, Y, N):
  inds = np.full((X.shape[0]), False)
  for i in range(N):
    inds = np.bitwise_or(inds, Y[:, i] == 1)
  X_new, Y_new = X[inds], Y[inds]
  for i in range(Y.shape[1], N, -1):
    Y_new = np.delete(Y_new, i-1, axis=1)
  return X_new, Y_new

# Training Data
X_train, Y_train = load_data(TRAIN_DIR, TRAIN_SUFFIX, SUBJECT_NUM)

# Test Data
X_test, Y_test = load_data(TEST_DIR, TEST_SUFFIX, SUBJECT_NUM)

# Validation Data
X_val, Y_val = load_data(VAL_DIR, VAL_SUFFIX, SUBJECT_NUM)

if (SHUFFLE):
  # Training Data
  p = np.random.permutation(X_train.shape[0])
  X_train = X_train[p]
  Y_train = Y_train[p]
  
  # Test Data
  p = np.random.permutation(X_test.shape[0])
  X_test = X_test[p]
  Y_test = Y_test[p]
  
  # Validation Data
  p = np.random.permutation(X_val.shape[0])
  X_val = X_val[p]
  Y_val = Y_val[p]

if (X_test.shape[-1] != X_train.shape[-1]):
  if (DIMENSION == '3D'):
    X_test = X_test[:, :, :, :X_train.shape[-1]]
  elif (DIMENSION == '2D'):
    X_test = X_test[:, :, :X_train.shape[-1]]

if (X_val.shape[-1] != X_train.shape[-1]):
  if (DIMENSION == '3D'):
    X_val = X_val[:, :, :, :X_train.shape[-1]]
  elif (DIMENSION == '2D'):
    X_val = X_val[:, :, :X_train.shape[-1]]

X_train, Y_train = select_classes(X_train, Y_train, NUM_CLASSES)
X_test, Y_test = select_classes(X_test, Y_test, NUM_CLASSES)
X_val, Y_val = select_classes(X_val, Y_val, NUM_CLASSES)

print("Shape of X Training Data:")
print(X_train.shape)
print("Shape of Y Training Data")
print(Y_train.shape)

print("Shape of X Test Data:")
print(X_test.shape)
print("Shape of Y Test Data")
print(Y_test.shape)

print("Shape of X Validation Data:")
print(X_val.shape)
print("Shape of Y Validation Data")
print(Y_val.shape)

Shape of X Training Data:
(3450, 7, 6, 313)
Shape of Y Training Data
(3450, 4)
Shape of X Test Data:
(1150, 7, 6, 313)
Shape of Y Test Data
(1150, 4)
Shape of X Validation Data:
(1160, 7, 6, 313)
Shape of Y Validation Data
(1160, 4)


# Models


## Model Definitions

### 3D Model Definition

In [ ]:
def EEGNet3D(nb_classes, XDim = 7, YDim = 6, Samples = 240, spatialSize=2, dropoutRate = 0.5, smallKernLength = 64, mediumKernLength = 96, largeKernLength = 160, F1 = 8, D = 2, F2 = 16, norm_rate = 0.15, dropoutType = 'Dropout', branches = ("SRF", "MRF", "LRF"), pooling1 = 4, pooling2 = 8, convSize = 16):
	if dropoutType == 'SpatialDropout3D':
		dropoutType = SpatialDropout3D
	elif dropoutType == 'Dropout':
		dropoutType = Dropout
	else:
		raise ValueError('dropoutType must be one of SpatialDropout3D or Dropout, passed as a string.')
    
	# This is dumb, but I can't think of a better way while allowing automatic hp tuning
	# Resolves the constraint that F2 must be a factor of the product of F1 and D
	# Not aware of a way to imply this relationship to KerasTuner parameter spaces
	f2_cands = []
	for i in range(1, D*F1+1):
		if ((D * F1) % i == 0):
			f2_cands.append(i)
	if (F2 >= len(f2_cands)):
		F2 = f2_cands[-1]
	else:
		F2 = f2_cands[F2]

	input1 = Input(shape = (XDim, YDim, Samples, 1))
	
	add_params = []
	if ("SRF" in branches):
		SRF_branch = EEGNet3D_Branch(nb_classes, XDim, YDim, Samples, spatialSize, dropoutRate, smallKernLength, F1, D, F2, norm_rate, dropoutType, pooling1, pooling2, convSize, input1)
		add_params.append(SRF_branch)
	if ("MRF" in branches):
		MRF_branch = EEGNet3D_Branch(nb_classes, XDim, YDim, Samples, spatialSize, dropoutRate, mediumKernLength, F1, D, F2, norm_rate, dropoutType, pooling1, pooling2, convSize, input1)
		add_params.append(MRF_branch)
	if ("LRF" in branches):
		LRF_branch = EEGNet3D_Branch(nb_classes, XDim, YDim, Samples, spatialSize, dropoutRate, largeKernLength, F1, D, F2, norm_rate, dropoutType, pooling1, pooling2, convSize, input1)
		add_params.append(LRF_branch)

	if (len(add_params) > 1):
		final = Add()(add_params)
	else:
		final = add_params[0]

	softmax = Activation('softmax', name = 'softmax')(final)
        
	return Model(inputs=input1, outputs=softmax)

def EEGNet3D_Branch(nb_classes, XDim, YDim, Samples, spatialSize, dropoutRate, kernLength, F1, D, F2, norm_rate, dropoutType, pooling1, pooling2, convSize, block):
	block1 = Conv3D(F1, (spatialSize, spatialSize, kernLength), padding = 'same', input_shape = (XDim, YDim, Samples, 1), use_bias = False)(block)
	block1 = BatchNormalization()(block1)
	block1 = Conv3D(D*F1, (XDim, YDim, 1), groups = F1, kernel_constraint = max_norm(1.), use_bias = False)(block1)
	block1 = BatchNormalization()(block1)
	block1 = Activation('elu')(block1)
	block1 = AveragePooling3D((1, 1, pooling1))(block1)
	block1 = dropoutType(dropoutRate)(block1)

	block2 = Conv3D(D*F2, (1, 1, convSize), groups = F2, use_bias = False, padding = 'same')(block1)
	block2 = Conv3D(F2, (1, 1, 1), use_bias = False, padding = 'same')(block2) 
	block2 = BatchNormalization()(block2)
	block2 = Activation('elu')(block2)
	block2 = AveragePooling3D((1, 1, pooling2))(block2)
	block2 = dropoutType(dropoutRate)(block2)

	flatten = Flatten()(block2)

	return Dense(nb_classes, kernel_constraint = max_norm(norm_rate))(flatten)


### 2D Model Definition

In [ ]:
def EEGNet2D(nb_classes, Channels = 22, Samples = 240, dropoutRate = 0.5, smallKernLength = 64, mediumKernLength = 96, largeKernLength = 160, F1 = 8, D = 2, F2 = 16, norm_rate = 0.25, dropoutType = 'Dropout', pooling1=4, pooling2=8, convSize=16, branches = ("SRF", "MRF", "LRF")):
  if dropoutType == 'SpatialDropout2D':
    dropoutType = SpatialDropout2D
  elif dropoutType == 'Dropout':
    dropoutType = Dropout
  else:
    raise ValueError('dropoutType must be one of SpatialDropout2D or Dropout, passed as a string.')

  input1 = Input(shape = (Channels, Samples, 1))

  add_params = []
  if ("SRF" in branches):
    SRF_branch = EEGNet2D_Branch(nb_classes, Channels, Samples, dropoutRate, smallKernLength, F1, D, F2, norm_rate, dropoutType, pooling1, pooling2, convSize, input1)
    add_params.append(SRF_branch)
  if ("MRF" in branches):
    MRF_branch = EEGNet2D_Branch(nb_classes, Channels, Samples, dropoutRate, mediumKernLength, F1, D, F2, norm_rate, dropoutType, pooling1, pooling2, convSize, input1)
    add_params.append(MRF_branch)
  if ("LRF" in branches):
    LRF_branch = EEGNet2D_Branch(nb_classes, Channels, Samples, dropoutRate, largeKernLength, F1, D, F2, norm_rate, dropoutType, pooling1, pooling2, convSize, input1)
    add_params.append(LRF_branch)

  if len(add_params) > 1:
    final = Add()(add_params)
  else:
    final = add_params[0]

  softmax = Activation('softmax', name = 'softmax')(final)
    
  return Model(inputs=input1, outputs=softmax)

def EEGNet2D_Branch(nb_classes, Channels, Samples, dropoutRate, kernLength, F1, D, F2, norm_rate, dropoutType, pooling1, pooling2, convSize, block):
  block1 = Conv2D(F1, (1, kernLength), padding = 'same', input_shape = (Channels, Samples, 1), use_bias = False)(block)
  block1 = BatchNormalization()(block1)
  # Could be swapped for depthwise conv
  block1 = DepthwiseConv2D((Channels, 1), use_bias=False, depth_multiplier=D, depthwise_constraint=max_norm(1.))(block1)
  #block1 = Conv2D(D*F1, (Channels, 1), groups = F1, kernel_constraint = max_norm(1.), use_bias = False)(block1)
  block1 = BatchNormalization()(block1)
  block1 = Activation('elu')(block1)
  block1 = AveragePooling2D((1, pooling1))(block1)
  block1 = dropoutType(dropoutRate)(block1)

  # Both conv layers could be swapped for a separable
  block2 = SeparableConv2D(F2, (1, convSize), use_bias=False, padding='same')(block1)
  #block2 = Conv2D(F2, (1, convSize), groups = F2, use_bias = False, padding = 'same')(block1)
  #block2 = Conv2D(F2, (1, 1), use_bias = False, padding = 'same')(block2) 
  block2 = BatchNormalization()(block2)
  block2 = Activation('elu')(block2)
  block2 = AveragePooling2D((1, pooling2))(block2)
  block2 = dropoutType(dropoutRate)(block2)

  flatten = Flatten()(block2)

  return Dense(nb_classes, kernel_constraint = max_norm(norm_rate))(flatten)


### 2D Model Simple Definition

In [ ]:
def EEGNet2D_Simple(nb_classes, Channels = 22, Samples = 240, dropoutRate = 0.5, smallKernLength = 64, mediumKernLength = 96, largeKernLength = 160, F1 = 8, D = 2, F2 = 16, norm_rate = 0.25, dropoutType = 'Dropout', pooling1=4, pooling2=8, convSize=16, branches = ("SRF", "MRF", "LRF")):
  if dropoutType == 'SpatialDropout2D':
    dropoutType = SpatialDropout2D
  elif dropoutType == 'Dropout':
    dropoutType = Dropout
  else:
    raise ValueError('dropoutType must be one of SpatialDropout2D or Dropout, passed as a string.')

  input1 = Input(shape = (Channels, Samples, 1))

  add_params = []
  if ("SRF" in branches):
    SRF_branch = EEGNet2D_Simple_Branch(nb_classes, Channels, Samples, dropoutRate, smallKernLength, F1, D, F2, norm_rate, dropoutType, pooling1, pooling2, convSize, input1)
    add_params.append(SRF_branch)
  if ("MRF" in branches):
    MRF_branch = EEGNet2D_Simple_Branch(nb_classes, Channels, Samples, dropoutRate, mediumKernLength, F1, D, F2, norm_rate, dropoutType, pooling1, pooling2, convSize, input1)
    add_params.append(MRF_branch)
  if ("LRF" in branches):
    LRF_branch = EEGNet2D_Simple_Branch(nb_classes, Channels, Samples, dropoutRate, largeKernLength, F1, D, F2, norm_rate, dropoutType, pooling1, pooling2, convSize, input1)
    add_params.append(LRF_branch)

  if len(add_params) > 1:
    branch_add = Add()(add_params)
  else:
    branch_add = add_params[0]

  block = AveragePooling2D((1, pooling1), name="Average_Pooling_1")(branch_add)
  block = dropoutType(dropoutRate)(block)
  block = SeparableConv2D(F2, (1, convSize), use_bias=False, padding='same')(block)
  block = BatchNormalization()(block)
  block = Activation('elu')(block)
  block = AveragePooling2D((1, pooling2))(block)
  block = dropoutType(dropoutRate)(block)
  block = Flatten()(block)
  block = Dense(nb_classes, kernel_constraint = max_norm(norm_rate))(block)

  softmax = Activation('softmax', name = 'softmax')(block)
    
  return Model(inputs=input1, outputs=softmax)

def EEGNet2D_Simple_Branch(nb_classes, Channels, Samples, dropoutRate, kernLength, F1, D, F2, norm_rate, dropoutType, pooling1, pooling2, convSize, block):
  block1 = Conv2D(F1, (1, kernLength), padding = 'same', input_shape = (Channels, Samples, 1), use_bias = False)(block)
  block1 = BatchNormalization()(block1)
  # Could be swapped for depthwise conv
  block1 = DepthwiseConv2D((Channels, 1), use_bias=False, depth_multiplier=D, depthwise_constraint=max_norm(1.))(block1)
  #block1 = Conv2D(D*F1, (Channels, 1), groups = F1, kernel_constraint = max_norm(1.), use_bias = False)(block1)
  block1 = BatchNormalization()(block1)
  block1 = Activation('elu')(block1)

  return block1

## Model Functions

### 3D Model

In [ ]:
def build_3d_model(hp):
  N = NUM_CLASSES
  Samples = DEFAULT_SAMPLES
  B = BRANCHES
  if (DROPOUT_RATE_3D is None):
    DR = hp.Float('Dropout Rate', 0.001, 0.999, sampling='linear')
  else:
    DR = DROPOUT_RATE_3D
  if (SRF_3D is None):
    SKL = hp.Int('Small Kernel Length', 16, 64)
  else:
    SKL = SRF_3D
  if (MRF_3D is None):
    MKL = hp.Int('Medium Kernel Length', 64, 128)
  else:
    MKL = MRF_3D
  if (LRF_3D is None):
    LKL = hp.Int('Large Kernel Length', 128, 192)
  else:
    LRL = LRF_3D
  if (F1_3D is None):
    F1 = hp.Int('Filter 1', 2, 16)
  else:
    F1 = F1_3D
  if (D_3D is None):
    D = hp.Int('Depth', 2, 4)
  else:
    D = D_3D
  if (F2_3D is None):
    F2 = hp.Int('Filter 2', 0, 4)
  else:
    F2 = F2_3D
  if (NORM_RATE_3D is None):
    NR = hp.Float('Normalization Rate', 0, 0.999, sampling='linear')
  else:
    NR = NORM_RATE_3D
  if (DROPOUT_TYPE_3D is None):
    DT = hp.Choice('Dropout Type', ('Dropout', 'SpatialDropout3D'))
  else:
    DT = DROPOUT_TYPE_3D
  if (SPATIAL_SIZE_3D is None):
    SS = hp.Int('Spatial Size', 1, 6)
  else:
    SS = SPATIAL_SIZE_3D
  if (POOLING_1_3D is None):
    P1 = hp.Int('Pooling 1', 2, 8)
  else:
    P1 = POOLING_1_3D
  if (CONV_SIZE_3D is None):
    CS = hp.Int('Convolution Size', 4, 32)
  else:
    CS = CONV_SIZE_3D
  if (POOLING_2_3D is None):
    P2 = hp.Int('Pooling 2', 2, 8)
  else:
    P2 = POOLING_2_3D
  if (INIT_LR is None):
    LR = hp.Float('Learning Rate', 0.000001, 0.1, sampling='log')
  else:
    LR = INIT_LR

  model = EEGNet3D(N, Samples=Samples, dropoutRate=DR, smallKernLength=SKL, mediumKernLength=MKL, largeKernLength=LKL, F1=F1, F2=F2, D=D, norm_rate=NR, dropoutType=DT, spatialSize=SS, pooling1=P1, pooling2=P2, convSize=CS, branches=B)
  Opt = OPTIMIZER(LR)
  model.compile(loss=LOSS_FUNCTION, optimizer=Opt, metrics=METRICS)
  return model

### 2D Model

In [ ]:
def build_2d_model(hp):
  N = NUM_CLASSES
  Samples = DEFAULT_SAMPLES
  B = BRANCHES
  if (DROPOUT_RATE_2D is None):
    DR = hp.Float('Dropout Rate', 0.001, 0.999, sampling='linear')
  else:
    DR = DROPOUT_RATE_2D
  if ("SRF" in BRANCHES):
    if (SRF_2D is None):
      SKL = hp.Int('Small Kernel Length', 2, 64)
    else:
      SKL = SRF_2D
  if ("MRF" in BRANCHES):
    if (MRF_2D is None):
      MKL = hp.Int('Medium Kernel Length', 64, 128)
    else:
      MKL = MRF_2D
  if ("LRF" in BRANCHES):
    if (LRF_2D is None):
      LKL = hp.Int('Large Kernel Length', 128, 313)
    else:
      LKL = LRF_2D
  if (F1_2D is None):
    F1 = hp.Int('Filter 1', 2, 16)
  else:
    F1 = F1_2D
  if (D_2D is None):
    D = hp.Int('Depth', 2, 4)
  else:
    D = D_2D
  if (F2_2D is None):
    F2 = hp.Int('Filter 2', 2, 32)
  else:
    F2 = F2_2D
  if (NORM_RATE_2D is None):
    NR = hp.Float('Normalization Rate', 0, 0.999, sampling='linear')
  else:
    NR = NORM_RATE_2D
  if (DROPOUT_TYPE_2D is None):
    DT = hp.Choice('Dropout Type', ('Dropout', 'SpatialDropout2D'))
  else:
    DT = DROPOUT_TYPE_2D
  if (POOLING_1_2D is None):
    P1 = hp.Int('Pooling 1', 2, 8)
  else:
    P1 = POOLING_1_2D
  if (CONV_SIZE_2D is None):
    CS = hp.Int('Convolution Size', 4, 64)
  else:
    CS = CONV_SIZE_2D
  if (POOLING_2_2D is None):
    P2 = hp.Int('Pooling 2', 2, 8)
  else:
    P2 = POOLING_2_2D
  if (INIT_LR is None):
    LR = hp.Float('Learning Rate', 0.0001, 0.001, sampling='log')
  else:
    LR = INIT_LR

  model = EEGNet2D_Simple(N, Samples=Samples, dropoutRate=DR, smallKernLength=SKL, mediumKernLength=MKL, largeKernLength=LKL, F1=F1, F2=F2, D=D, norm_rate=NR, dropoutType=DT, pooling1=P1, pooling2=P2, convSize=CS, branches=B)
  Opt = OPTIMIZER(LR)
  model.compile(loss=LOSS_FUNCTION, optimizer=Opt, metrics=METRICS)

  return model

# KerasTuner

## Custom Hyper Model
Allows for Batch Size to be learned

In [ ]:
class CustomHyperModel(kt.HyperModel):
  def build(self, hp):
    if (BATCH_SIZE is None):
      self.BS = hp.Int('Batch Size', 4, 32)
    else:
      self.BS = BATCH_SIZE
    
    if (DIMENSION == '3D'):
      return build_3d_model(hp)
    return build_2d_model(hp)
    
  def fit(self, hp, model, *args, **kwargs):
    return model.fit(*args, batch_size=self.BS, **kwargs)

# Tuning Runtime

In [ ]:
from keras_tuner.tuners.bayesian import BayesianOptimization

OBJECTIVE = 'val_Accuracy'
tuner = BayesianOptimization(
    CustomHyperModel(),
    objective=OBJECTIVE,
    max_trials=NUM_ITERATIONS,
    directory=TUNING_DIRECTORY,
    project_name='KerasTuner-Bayesian-{}{}-{}-{}-{}-{}-{}'.format(PROJECT_TAG, SUBJECT_NUM, NUM_CLASSES, MODE, DIMENSION, BRANCHED, OBJECTIVE),
    overwrite=OVERWRITE
)

tuner.search_space_summary()

if (SEARCH):
  tuner.search(X_train, Y_train, epochs=EPOCHS, validation_data=(X_val, Y_val), callbacks=CALLBACKS, verbose=VERBOSE)

Trial 36 Complete [01h 39m 00s]
val_Accuracy: 0.3353448212146759

Best val_Accuracy So Far: 0.642241358757019
Total elapsed time: 23h 51m 27s

Search: Running Trial #37

Value             |Best Value So Far |Hyperparameter
19                |29                |Batch Size
64                |64                |Small Kernel Length
86                |70                |Medium Kernel Length
160               |151               |Large Kernel Length
8                 |7                 |Pooling 1
32                |21                |Convolution Size
4                 |8                 |Pooling 2
0.1               |0.00019897        |Learning Rate

Epoch 1/500
182/182 [==============================] - 21s 102ms/step - loss: 1.4581 - Accuracy: 0.2633 - Precision: 0.3707 - Recall: 0.0165 - val_loss: 1.4313 - val_Accuracy: 0.2500 - val_Precision: 0.0000e+00 - val_Recall: 0.0000e+00
Epoch 2/500
182/182 [==============================] - 18s 97ms/step - loss: 1.4298 - Accuracy: 0.2586 - Precisio

In [ ]:
tuner.results_summary()